In [8]:
import numpy as np
import matplotlib.pyplot as plt
import scipy as sp

import DMsimulatorCO as DMsim
from DMoptimizeCO import Optimize

In [9]:

#########* Constants and fixed parameters of the model
const_dict = {
        "F0": 1.5e15,           # cm^-2
        "S0": 3e13,             # cm^-2
        
        "R": 0.00831442,        # kJ/mol*K
        "kBoltz": 1.380649e-23, # J/K
}

#########* Steric factors reactions
steric_O_dict = {
        ###* Atomic oxygen
        "SF_O_F": 1.0, "SF_O_S": 1.0, "SF_O_SO": 1.0, "SF_O_FO": 1.0,
        "SF_FO_S": 1.0, "SF_FO_SO": 1.0, "SF_FO_FO": 1.0, "SF_FO": 1.0,
        
        ###* Molecular oxygen
        "SF_O2_F": 1.0, "SF_O2_FO": 1.0, "SF_O2_FO2": 1.0, "SF_O_FO2": 1.0,
        "SF_FO2_FO": 1.0, "SF_FO_FO2": 1.0, "SF_FO2": 1.0,
        
        ###* Metastable species
        "SF_O2fast_SO": 1.0*1.0, "SF_Ofast_SO": 0.0*1.0, "SF_O2fast_S": 1.0*1.0,  "SF_Ofast_S": 0.0*1.0,
        "SF_Ofast_Sdb": 1.0*1.0, "SF_Ofast_SOdb": 1.0*1.0, "SF_O2fast_Sdb": 1.0*1.0, "SF_O2fast_SOdb": 1.0*1.0,
        "SF_O_Sdb": 1.0*1.0, "SF_O_SOdb": 1.0*1.0, "SF_FO_SOdb": 1.0*1.0, "SF_FO_Sdb": 1.0*1.0,
}

steric_CO_dict = {
        ###* CO2 specie
        "SF_CO2_F": 0.0*1.0, "SF_FCO2": 0.0*1.0,
        
        ###* CO specie
        "SF_CO_F": 0.0*1.0, "SF_FCO": 0.0*1.0, "SF_CO_S": 0.0*1e-2, "SF_O_FCO": 0.0*1e-1,
        "SF_CO_FO": 0.0*1e-1, "SF_FCO_FO": 0.0*1e-1, "SF_FCO_S": 0.0*1e-2, "SF_O_SCO": 0.0*1e-1,
        "SF_CO_SO": 0.0*1e-1, "SF_FO_SCO": 0.0*1e-1, "SF_FCO_SO": 0.0*1e-1,
        
        ###* Metastable species
        "SF_COfast_SO": 0.0, "SF_CO2fast_SO": 0.0, "SF_Ofast_SCO": 0.0*1.0, "SF_O2fast_SCO": 0.0*1.0,
        "SF_COfast_SCO": 0.0, "SF_CO2fast_SCO": 0.0, "SF_COfast_S": 0.0, "SF_CO2fast_S": 0.0,
        "SF_CO_Sdb": 0.0*1e-2, "SF_CO_Sdb_2": 0.0*1.0, "SF_CO2_Sdb": 0.0*1.0, "SF_CO2_SOdb": 0.0*1.0,
        "SF_CO_SOdb": 0.0*1e-1, "SF_O_SCOdb": 0.0*1e-1, "SF_CO_SCOdb": 0.0*1.0, "SF_CO2_SCOdb": 0.0*1.0,
        
        "SF_O2_SCOdb": 1.0, "SF_CO_SOdb": 0.0*1e-1, "SF_O_SCOdb": 0.0*1e-1, "SF_FCO_SOdb": 0.0*1e-2,
        "SF_FCO_SCOdb": 0.0*1e-1, "SF_FO_SCOdb": 0.0*1e-1
}


#########* Energy barriers  --  Based Model

energy_O_dict = { # kJ/mol and s^-1
        "nu_D_O": 1.0e13, "nu_d_O": 1.0e15,
        
        ###* Atomic oxygen
        "E_O_F": 0.0, "E_O_S": 0.0, "E_O_SO": 15.0, "E_O_FO": 0.0, 
        "E_FO_SO": 20.0, "E_FO_FO":0.0, "E_di_O": 15.0, "E_de_O": 30.0,
        
        ###* Molecular oxygen
        "E_O2_F": 0.0, "E_O2_FO": 0.0, "E_O2_FO2": 0.0, "E_O_FO2": 0.0, 
        "E_FO2_FO": 0.0, "E_FO_FO2": 0.0, "E_di_O2": 15.0, "E_de_O2": 17.5,
        
        ###* Metastable species
        "E_O2fast_SO": 0.0, "E_O2fast_S": 0.0, "E_O2fast_SOdb": 0.0, "E_O2fast_Sdb": 0.0, "E_Ofast_Sdb": 0.0,
        "E_Ofast_SOdb": 0.0, "E_O_Sdb": 0.0, "E_O_SOdb": 0.0, "E_F_SOdb": 0.0, "E_FO_SOdb": 0.0,
        "ED_db": 14.999,
        
        "Emin": 2.90, # eV
        "Ealpha": 3400.0, # K
}

energy_CO_dict = { # kJ/mol and s^-1
        "nu_D_CO": 1.0e13, "nu_d_CO": 1.0e15,
        
        ###* CO2 specie
        "E_CO2_F": 0.0, "E_FCO2": 17.5,
        
        ###* CO specie
        "E_CO_F": 0.0, "E_FCO": 17.5, "E_CO_S": 0.0, "E_O_FCO": 0.0,
        "E_CO_FO": 0.0, "E_FCO_FO": 29.0, "E_FCO_S": 29.0, "E_O_SCO": 29.0,
        "E_CO_SO": 29.0, "E_FO_SCO": 49.0, "E_FCO_SO": 49.0,
        
        ###* Metastable species
        "E_di_CO": 15.0, "E_de_CO": 15.0, # kJ/mol 
        "E_COfast_SO": 45.0, "E_CO2fast_SO": 45.0, "E_Ofast_SCO": 45.0, "E_O2fast_SCO": 45.0,
        "E_COfast_SCO": 45.0, "E_CO2fast_SCO": 45.0, "E_COfast_S": 45.0, "E_CO2fast_S": 45.0,
        "E_CO_Sdb": 0.0, "E_CO_Sdb_2": 0.0, "E_CO2_Sdb": 0.0, "E_CO2_SOdb": 0.0,
        "E_CO_SOdb": 0.0, "E_O_SCOdb": 0.0, "E_CO_SCOdb": 0.0, "E_CO2_SCOdb": 0.0,
        
        "E_O2_SCOdb": 0.0, "E_CO_SOdb": 0.0, "E_O_SCOdb": 0.0, "E_FCO_SOdb": 30.0,
        "E_FCO_SCOdb": 30.0, "E_FO_SCOdb": 30.0,
}


file_input_data = "Experimental_data_Paper.hdf5"


In [10]:

steric_dict = (steric_O_dict, steric_CO_dict)

system  = DMsim.SurfaceKineticsSimulator(const_dict, steric_dict, file_input_data, 
                                        init_conditions=[0.1, 0.1, 0.6, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])

input_experimental_data, recomb_exp_vec = system.prepare_data()

print(input_experimental_data)
print(recomb_exp_vec.shape)


[0.1, 0.1, 0.6, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]
[{'pressure': 0.4, 'current': 20.0, 'Tnw': 262.53133733208, 'Tw': 253.14999999999998, 'O_den': 1493906152913230.0, 'CO_den': 0.0, 'N_den': 1.34810445111836e+16, 'FluxIon': 2000000000000000.0, 'EavgMB': 0.87}
 {'pressure': 0.6, 'current': 20.0, 'Tnw': 265.49039366168, 'Tw': 253.14999999999998, 'O_den': 2482489172052840.0, 'CO_den': 0.0, 'N_den': 1.9502568519941e+16, 'FluxIon': 2000000000000000.0, 'EavgMB': 0.77}
 {'pressure': 0.8, 'current': 20.0, 'Tnw': 268.28248792299996, 'Tw': 253.14999999999998, 'O_den': 3044362383817350.0, 'CO_den': 0.0, 'N_den': 2.51593341846723e+16, 'FluxIon': 2000000000000000.0, 'EavgMB': 0.4667333333333333}
 {'pressure': 1.5, 'current': 20.0, 'Tnw': 276.84558188536, 'Tw': 253.14999999999998, 'O_den': 4814831897137840.0, 'CO_den': 0.0, 'N_den': 4.29026203257012e+16, 'FluxIon': 2000000000000000.0, 'EavgMB': 0.001}
 {'pressure': 2.0, 'current': 20.0, 'Tnw': 281.93841257599996, 'Tw': 253.14999999999998, 'O_den': 5660345

In [11]:

result_gammas = []
result_names = []

energy_dict = energy_O_dict.copy()
energy_dict.update(energy_CO_dict)

for i in range(len(input_experimental_data)):
    
    exp_dict = input_experimental_data[i]
    _, gammas, gammas_names, sucess = system.solve_system(exp_dict, energy_dict)
    
    if sucess == False:
        print(f"Simulation failed for index {i}. Assigning high loss.")
    
    result_gammas.append(gammas)

result_names.append(gammas_names)



0.4 20.0 262.53133733208 253.14999999999998
{'FluxO': 2.2013036078833943e+19, 'FluxO2': 1.2489847901030236e+20, 'FluxIon': 2000000000000000.0}
O:  {'r1': 4549.839027226432, 'r2': 645547731.6715667, 'r3': 14387.605280283622, 'r4': 14387.605280283622, 'r5': 6025948880.178593, 'r6': 14.912525716375674, 'r7': 600136.1013818666, 'r8': 8034598506.90479}
O2:  {'r9': 81632.99281719109, 'r10': 244976754281.27188, 'r11': 81632.99281719109, 'r12': 81632.99281719109, 'r14': 14387.605280283622, 'r15a': 8034598506.90479, 'r15b': 0.0}
meta:  {'r16': 0.023200144409936205, 'r18': 0.023200144409936205, 'r20': 14387.461398683088, 'r21': 0.14388160053520113, 'r22': 0.8163614051263779, 'r23': 0.8163614051263779, 'r24': 0.14388160053520113, 'r25': 14387.461398683088, 'r26': 4843908.808915501, 'r27': 4843908.808915501}
frac_Of, frac_O2f, frac_Os, frac_Osdb, frac_Svdb, frac_COf, frac_CO2f, frac_COs, frac_COsdb
[7.04507860e-06 3.33224840e-07 9.76080185e-01 1.17949736e-02
 1.17957590e-02 0.00000000e+00 0.000000

In [12]:

result_gammas_arr = np.array(result_gammas)
print(result_gammas_arr.shape)

gammas_tot = np.sum(result_gammas_arr, axis=-1)
print(gammas_tot.shape)

(66, 29)
(66,)


In [13]:
print(result_gammas_arr[:5])

[[1.38138796e-05 3.96741844e-05 1.12484426e-05 5.43472000e-05
  3.91888821e-05 1.85359308e-06 3.26691019e-07 1.28528282e-06
  2.62452518e-08 4.62565821e-09 4.62543360e-04 1.09710977e-06
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00]
 [2.31977221e-05 4.34416113e-05 1.14645944e-05 9.17146359e-05
  5.62308392e-05 2.26138247e-06 4.66460422e-07 1.84420038e-06
  1.05767713e-08 2.18169429e-09 1.89822498e-04 4.52457466e-07
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00]
 [2.87284392e-05 4.70465878e-05 1.16212573e-05 1.14101482e-04
  7.37831310e-05 3.12957245e-06 6.

In [14]:
print(gammas_tot)

[0.00062541 0.00042091 0.00029538 0.00044697 0.0005449  0.00066941
 0.00087783 0.0005388  0.00033056 0.00038616 0.00050546 0.00060961
 0.00076959 0.00099965 0.00121026 0.00042813 0.00026691 0.00016415
 0.00017249 0.00020978 0.00024303 0.00029758 0.00037499 0.00063223
 0.0003595  0.00018999 0.00019789 0.00024207 0.00028352 0.00035282
 0.0004609  0.00056658 0.00038259 0.00024995 0.00016394 0.00016715
 0.00018989 0.00021026 0.00024342 0.00029696 0.00056269 0.0003344
 0.00018823 0.00018925 0.00021888 0.00024792 0.0003017  0.00039461
 0.00048218 0.00037961 0.00027617 0.00020861 0.00021178 0.00023185
 0.00024968 0.00027854 0.000322   0.00054028 0.00035489 0.00023448
 0.00023665 0.00026525 0.00029409 0.00034984 0.00044965 0.00054055]
